In [84]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import pdb
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
import os
#change to working directory
os.chdir(r"C:\Users\drusi\Kaggle\Prosumer\predict-energy-behavior-of-prosumers")



In [87]:
from datetime import datetime

def datestr_to_int(datetime_str, date_format):
    try:
        # Try parsing with timezone information
        dt = datetime.strptime(datetime_str, date_format + '%z')
    except ValueError:
        # Fallback to parsing without timezone information
        dt = datetime.strptime(datetime_str, date_format)
    
    # Convert to timestamp
    timestamp = dt.timestamp()
    
    return timestamp

In [92]:
def merged_data():
    print("loading train data...")
    #load the training data, dropping NaN's
    train = pd.read_csv("train.csv").dropna()
    train['datetime'] = train['datetime'].apply(lambda x: datestr_to_int(x,'%Y-%m-%d %H:%M:%S'))
    #shift data_block_id by +1 to line up with electricity_prices and gas_prices
    train['data_block_id'] += 1

    print("Loading client data")
    client = pd.read_csv("client.csv")
    client['date'] = client['date'].apply(lambda x: datestr_to_int(x,'%Y-%m-%d'))
    client["data_block_id"] += 2
    
    print("loading gas_prices...")
    #load gas_prices
    gas_prices = pd.read_csv("gas_prices.csv")
    #convert date strings to ints
    gas_prices['forecast_date'] = gas_prices['forecast_date'].apply(lambda x: datestr_to_int(x,'%Y-%m-%d'))
    gas_prices = gas_prices.drop(columns=['origin_date'])

    print("loading electricity_prices...")
    #load electricity_prices
    electricity_prices = pd.read_csv("electricity_prices.csv")
    #convert date strings to ints
    electricity_prices['forecast_date'] = electricity_prices['forecast_date'].apply(lambda x: datestr_to_int(x,'%Y-%m-%d %H:%M:%S'))
    electricity_prices = electricity_prices.drop(columns=['origin_date'])

    print("loading forecast_weather...")
    #load forecast_weather
    forecast_weather = pd.read_csv("forecast_weather.csv")
    #convert strings to ints
    forecast_weather['forecast_datetime'] = forecast_weather['forecast_datetime'].apply(lambda x: datestr_to_int(x,'%Y-%m-%d %H:%M:%S'))
    forecast_weather = forecast_weather.drop(columns=['origin_datetime'])
    forecast_weather = forecast_weather.rename(columns={'forecast_datetime':'forecast_date'})
    #shift times to line up with gas/electricity
    forecast_weather['forecast_date'] -= 10_800

    print("loading historical_weather...")
    #load forecast_weather
    historical_weather = pd.read_csv("historical_weather.csv")
    #convert strings to ints
    historical_weather['datetime'] = historical_weather['datetime'].apply(lambda x: datestr_to_int(x,'%Y-%m-%d %H:%M:%S'))
    #shift times to line up with gas/electricity

    #merge all the data

    print("Merging train and client")
    # Perform the merge
    df = pd.merge(train, client[['product_type', 'is_business', 'county', 'data_block_id', 'eic_count', 'installed_capacity']], 
               on=['product_type', 'is_business', 'county', 'data_block_id'], 
               how='left')
    print("merging train and gas_prices...")
    #merge gas prices and train.csv data
    #column names differ, so use left_on and right_on
    df = pd.merge(df, gas_prices, left_on=['data_block_id','datetime'], right_on=['data_block_id','forecast_date'], how='left')

    print("merging electricity_prices...")
    #merge train and gas_prices via left join on data_block_id
    #this leaves all rows of train, but matches
    df = df.merge(electricity_prices, on=['data_block_id','forecast_date'], how='left')

    print("merging forecast_weather...")
    #merge forecast_weather on forecast date
    df = df.merge(forecast_weather, on=['data_block_id','forecast_date'],how='left')

    print("merging historical_weather...")
    #merge historical_weather on datetime
    df = df.merge(historical_weather, left_on=['data_block_id','forecast_date'], right_on=['data_block_id','datetime'], how='left')

    #rename datetime to prediction datetime
    df = df.rename(columns={'datetime': 'prediction_datetime'})

    #drop NaN rows
    df = df.dropna()

In [ ]:
df1 = merged_data()


loading train data...
Loading client data
loading gas_prices...
loading electricity_prices...
loading forecast_weather...
loading historical_weather...
Merging train and client data
merging train and gas_prices...
merging electricity_prices...
merging forecast_weather...
merging historical_weather...


In [68]:


# Assuming 'target' is your target column
X = df1.drop('target', axis=1)
y = df1['target']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the XGBoost regressor
model = XGBRegressor()

# Perform cross-validation
cv_scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_absolute_error')

# Train the model on the entire training set
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model using Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error on test set: {mae}')

# Display cross-validation scores
print("Cross-Validation Scores:")
for i, score in enumerate(cv_scores, 1):
    print(f'Fold {i}: {abs(score)}')

# Calculate and display the mean cross-validation score
mean_cv_score = abs(cv_scores.mean())
print(f'Mean Cross-Validation Score: {mean_cv_score}')



Mean Absolute Error on test set: 17.558620695370116
Cross-Validation Scores:
Fold 1: 29.86328148850852
Fold 2: 37.428999947813715
Fold 3: 39.26751645321682
Fold 4: 36.037484107643685
Fold 5: 53.72236923102057
Mean Cross-Validation Score: 39.26393024564066


In [ ]:
with open(r'D:\Kaggle\Prosumer\models\xgboost_model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

In [ ]:
import pandas as pd

# Assuming 'df' is your DataFrame with the provided data
df['prediction_unit_id'] = df.apply(lambda row: hash(tuple(row[['county', 'is_business', 'product_type']])), axis=1)

# Load the 'train.csv' file into another DataFrame
train_df = pd.read_csv('train.csv')

# Merge on 'prediction_unit_id' using a left merge
merged_df = pd.merge(train_df, df, on='prediction_unit_id', how='left')

# Display the merged DataFrame
print(merged_df)

